---
title: Subspace Iteration
description: Advanced techniques for improved accuracy and robustness in randomized low-rank approximation
keywords: [subspace iteration, block Krylov, power iteration, Krylov subspace, iterative methods, spectral gap, convergence acceleration]
numbering:
  equation:
    enumerator: 5.%s
    continue: true
  proof:theorem:
    enumerator: 5.%s
    continue: true
  proof:algorithm:
    enumerator: 5.%s
    continue: true
  proof:definition:
    enumerator: 5.%s
    continue: true
  proof:proposition:
    enumerator: 5.%s
    continue: true
  figure:
    enumerator: 5.%s
    continue: true
---



The [Randomized SVD](#alg-randomized-svd) (RSVD) produces an approximation
\begin{equation}
\widehat{\vec{A}} = \vec{Q}\vec{Q}^\T \vec{A},
\quad 
\vec{Q} = \Call{orth}(\vec{A}\vec{\Omega}),
\end{equation}
where $\vec{\Omega}\sim \operatorname{Gaussian}(n,b)$.
Ideally $\vec{Q}$ is well-aligned with the dominant subspace of $\vec{A}$. 
However, the approximation can be poor if the singular values of $\vec{A}$ decay slowly or if the spectral gap is small.

One way to mitigate this, is to damp down the tail of $\vec{A}$ relative to the leading singular values. 
In particular, observe that if $\vec{A}$ as (thin) SVD $\vec{A} = \vec{U}\vec{\Sigma}\vec{V}^\T$, 
\begin{equation}
(\vec{A}\vec{A}^\T)^q\vec{A} = \vec{U} \vec{\Sigma}^{2q+1} \vec{V}^\T.
\end{equation}
The singular values of $(\vec{A}\vec{A}^\T)^q\vec{A}$ are the singular values of $\vec{A}$ raised to the power $2q+1$.
Thus, as illustrated in {eq}`fig-SI` the small singular values become smaller relative to the large ones.

:::{figure}
:label: fig-si
:alt: Four subplots showing singular value decay for different powers q. Original singular values (q=0) decay gradually from 1.0 to 0.1. For q=1,2,4, the singular values are raised to powers 3,5,9 respectively, showing increasingly steep decay where small singular values become much smaller relative to large ones.
Observe that the singular values tail of $(\vec{A}\vec{A}^\T)^q\vec{A}$ is damped substantially relative to that of $\vec{A}$.

![Four subplots showing singular value decay for different powers q, demonstrating how small singular values are damped relative to large ones as q increases](./SI.svg)
:::


This leads to the *Randomized Subspace Iteration* (RSI) approximation:
\begin{equation}
\widehat{\vec{A}} = \vec{Q}\vec{Q}^\T \vec{A},
\quad 
\vec{Q} = \Call{orth}((\vec{A}\vec{A}^\T)^q\vec{A}\vec{\Omega}).
\end{equation}
Observe that $(\vec{A}\vec{A}^\T)^q\vec{A}\vec{\Omega}$ can be computed by sequential products with $\vec{A}$ and $\vec{A}^\T$.
In particular, we never need to form the (potentially large) matrix $\vec{A}\vec{A}^\T$ explicitly.



:::{prf:algorithm} Randomized Subspace Iteration
:label: alg-RSI

**Input:** Matrix $A \in \R^{m \times n}$, target rank $k$, iteratoni bound $q$, block-size $b\geq k$

1. Sample a random sketching matrix $\vec{\Omega}\sim\Call{Sketch}(n,b)$
1. Compute $\vec{Y} = \vec{A} \vec{\Omega}$
1. For $i=1,\ldots,q$:
   - Compute $\vec{Y} = \vec{A}^\T \vec{Y}$
    - Compute $\vec{Y} = \vec{A} \vec{Y}$
1. Compute QR factorization $\vec{Q}\vec{R} = \Call{qr}(\vec{Y})$
1. Compute $\vec{X} = \vec{A}^\T \vec{Q}$

**Output:** $\vec{Q}\vec{X}^\T$
:::

Note that for numerical stability reasons, it is often recommended to re-orthogonalize $\vec{Y}$ after each multiplication by $\vec{A}$ or $\vec{A}^\T$.

In [14]:
import numpy as np
import matplotlib.pyplot as plt

d = 20
Λ = np.sort(np.linspace(0,.9,d) + 1e-1*np.random.rand(d))[::-1]
Λ[0] = 1


fig,axs = plt.subplots(1,4,figsize=(8,4),sharey=True)

qs = [1,2,4]
axs[0].plot(np.arange(d),Λ,ls='None',marker='o',label='original singular values')
axs[0].set_title('original ($q=0$)')
axs[0].set_xlabel('index $i$')

for i,q in enumerate(qs):
    ax = axs[i+1]

    ax.plot(np.arange(d),Λ**(2*q+1),ls='None',marker='o',label=r'$\sigma_i(A)^{2q+1}$')

    ax.set_title(f'$q={q}$')

    ax.set_xlabel('index $i$')

axs[0].set_ylabel('singular value')
plt.savefig('SI.svg')